# Question 2

**Quel était le classement des pilotes à l'issue de la saison 2023 ? Gérer les cas d'égalité (nombre de victoires, deuxièmes places, etc.).**

In [ ]:
import pandas as pd

drivers = pd.read_csv('../data/drivers.csv')
results = pd.read_csv('../data/results.csv')
races = pd.read_csv('../data/races.csv')


0        10.0
1         8.0
2         6.0
3         5.0
4         4.0
         ... 
26514     0.0
26515     0.0
26516     0.0
26517     0.0
26518     0.0
Name: points, Length: 26519, dtype: float64

## Résolution

D'abord on regarde la 'positionOrder' (le rang final d'un pilote). Cependant il y a beaucoup d'égalités. On va également combiner avec le nombre de points.

Puisque l'on a le choix de la méthode de classement, je choisis de créer une colonne `rating` qui est une combinaison de la positionOrder et du nombre de points, et du rang. On va ensuite trier par `rating` pour obtenir le classement final.

In [ ]:
# 1. On ne récupère que les courses de 2023
results2023 = pd.merge(results,races,on = 'raceId').query('year == 2023')

results_join_drivers = pd.merge(drivers,results2023, on='driverId')[["driverId","forename", "surname", "positionOrder","points","rank"]].sort_values(by='points', ascending=False)

# 2. On groupe en sommant les points et en prenant la moyenne de positionOrder
grouped_results = results_join_drivers.groupby('driverId').agg({
    'points': 'sum',
    'positionOrder': 'mean',
    'forename': 'first',
    'surname': 'first',
    'rank': 'first'
}).reset_index()


# 3. On calcule le rating à l'aide de cette formule
grouped_results['rating'] = grouped_results['points'] - (grouped_results['positionOrder'] * 5) - (grouped_results['rank'].astype(int) * 10)

# 4. On normalise le rating entre 0 et 100 c'est plus sympa
min_rating = grouped_results['rating'].min()
max_rating = grouped_results['rating'].max()
grouped_results['rating'] = 100 * (grouped_results['rating'] - min_rating) / (max_rating - min_rating)
grouped_results['rating'] = grouped_results['rating'].round(1)

#5. On s'assure qu'il n'y a aucune égalité en vérifiant que le nombre de valeurs uniques est égal au nombre de lignes
assert grouped_results['rating'].nunique() == grouped_results['rating'].shape[0]

# 6. On affiche pour finir ⭐
grouped_results[["forename","surname","rating"]].sort_values(by='rating', ascending=False)

,forename,surname,rating
7,Max,Verstappen,100.0
3,Sergio,Pérez,55.8
0,Lewis,Hamilton,55.2
1,Fernando,Alonso,52.4
13,Lando,Norris,49.6
8,Carlos,Sainz,41.2
12,Charles,Leclerc,40.3
14,George,Russell,38.0
19,Oscar,Piastri,31.5
10,Lance,Stroll,25.5
